# TABULATING THE CONCENTRIC HIERARCHY

Since the Python ecosystem provides some excellent tabulation tools, so why not take advantage and build a pandas DataFrame showing alternative coordinate system treatments of the same vertexes (vertices) of a set of nested polyhedrons? 

But then vertices are not sufficent to define a polyhedron: we need faces, from which edges might be distilled. The faces to edges distallation process is easily implemented by algorithm.

These face graphs might be tabulated in matrix format, with the same vertexes down the side and across the top. To every row and column label there corresponds a vector tail orginating at the origin, the center of our concentric hiearechy.

In [1]:
import sys
sys.version

'3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]'

In [2]:
import tetravolume as tv
import flextegrity as fx
import pandas as pd
import numpy as np
from importlib import reload
import sympy

In [3]:
fx.Tetrahedron

flextegrity.Tetrahedron

In [4]:
import qrays
import flextegrity

In [5]:
reload(flextegrity)

<module 'flextegrity' from '/Users/kirbyurner/Documents/School_of_Tomorrow/flextegrity.py'>

In [6]:
fx.Tetrahedron().vertexes

{'a': ivm_vector(a=1, b=0, c=0, d=0),
 'b': ivm_vector(a=0, b=1, c=0, d=0),
 'c': ivm_vector(a=0, b=0, c=1, d=0),
 'd': ivm_vector(a=0, b=0, c=0, d=1)}

In [7]:
type(fx.Tetrahedron().vertexes['a'].b)

sympy.core.numbers.Zero

In [8]:
type(fx.A.b)

sympy.core.numbers.Zero

In [9]:
arrays = [
    coordsys := ["IVM", "IVM", "IVM", "IVM", "XYZ", "XYZ", "XYZ"],
    coordnms := ["A", "B", "C", "D", "X", "Y", "Z"]]
col_tuples = list(zip(*arrays))
col_tuples

[('IVM', 'A'),
 ('IVM', 'B'),
 ('IVM', 'C'),
 ('IVM', 'D'),
 ('XYZ', 'X'),
 ('XYZ', 'Y'),
 ('XYZ', 'Z')]

In [10]:
the_shape = fx.Cuboctahedron()
vs = the_shape.vertexes.items()
shape_name = the_shape.nick
rows = []
row_tuples = []

for label, v in vs:
    xyz = v.xyz
    rows.append(dict(name = label, A=v.a, B=v.b, C=v.c, D = v.d, X = xyz.x, Y = xyz.y, Z = xyz.z))
    row_tuples.append((shape_name, label))

In [11]:
row_tuples

[('CO', 'o'),
 ('CO', 'p'),
 ('CO', 'q'),
 ('CO', 'r'),
 ('CO', 's'),
 ('CO', 't'),
 ('CO', 'u'),
 ('CO', 'v'),
 ('CO', 'w'),
 ('CO', 'x'),
 ('CO', 'y'),
 ('CO', 'z')]

In [12]:
df = pd.DataFrame.from_dict(rows)
col_index = pd.MultiIndex.from_tuples(col_tuples, names=["System", "Coords"])
row_index = pd.MultiIndex.from_tuples(row_tuples, names=["Shape", "Verts"])
df.index = row_index
df.drop(columns="name", axis=1, inplace=True)
df.columns = col_index
df

System      IVM                  XYZ                        
Coords        A  B  C  D           X           Y           Z
Shape Verts                                                 
CO    o       2  1  1  0           0   sqrt(2)/2   sqrt(2)/2
      p       2  1  0  1   sqrt(2)/2           0   sqrt(2)/2
      q       1  2  1  0  -sqrt(2)/2           0   sqrt(2)/2
      r       1  2  0  1           0  -sqrt(2)/2   sqrt(2)/2
      s       1  0  2  1           0   sqrt(2)/2  -sqrt(2)/2
      t       1  0  1  2   sqrt(2)/2           0  -sqrt(2)/2
      u       0  1  2  1  -sqrt(2)/2           0  -sqrt(2)/2
      v       0  1  1  2           0  -sqrt(2)/2  -sqrt(2)/2
      w       1  1  2  0  -sqrt(2)/2   sqrt(2)/2           0
      x       0  2  1  1  -sqrt(2)/2  -sqrt(2)/2           0
      y       1  1  0  2   sqrt(2)/2  -sqrt(2)/2           0
      z       2  0  1  1   sqrt(2)/2   sqrt(2)/2           0

In [13]:
type(df.loc[('CO','o')].IVM.A)

sympy.core.numbers.Integer

In [14]:
df

System      IVM                  XYZ                        
Coords        A  B  C  D           X           Y           Z
Shape Verts                                                 
CO    o       2  1  1  0           0   sqrt(2)/2   sqrt(2)/2
      p       2  1  0  1   sqrt(2)/2           0   sqrt(2)/2
      q       1  2  1  0  -sqrt(2)/2           0   sqrt(2)/2
      r       1  2  0  1           0  -sqrt(2)/2   sqrt(2)/2
      s       1  0  2  1           0   sqrt(2)/2  -sqrt(2)/2
      t       1  0  1  2   sqrt(2)/2           0  -sqrt(2)/2
      u       0  1  2  1  -sqrt(2)/2           0  -sqrt(2)/2
      v       0  1  1  2           0  -sqrt(2)/2  -sqrt(2)/2
      w       1  1  2  0  -sqrt(2)/2   sqrt(2)/2           0
      x       0  2  1  1  -sqrt(2)/2  -sqrt(2)/2           0
      y       1  1  0  2   sqrt(2)/2  -sqrt(2)/2           0
      z       2  0  1  1   sqrt(2)/2   sqrt(2)/2           0

In [15]:
def tabulate(shape, dec=False, n=8):
    vs = shape.vertexes.items()
    shape_name = shape.nick
    rows = []
    row_tuples = []

    for label, v in vs:
        xyz = v.xyz
        row_tuples.append((shape_name, label))
        if not dec:
            rows.append(dict(name = label, 
                             A=v.a, 
                             B=v.b, 
                             C=v.c, 
                             D = v.d, 
                             X = xyz.x, 
                             Y = xyz.y, 
                             Z = xyz.z))
        else:
            rows.append(dict(name = label, 
                             A = v.a.evalf(n).round(n), 
                             B = v.b.evalf(n).round(n), 
                             C = v.c.evalf(n).round(n), 
                             D = v.d.evalf(n).round(n), 
                             X = xyz.x.evalf(n).round(n), 
                             Y = xyz.y.evalf(n).round(n), 
                             Z = xyz.z.evalf(n).round(n)))  
                        
    df = pd.DataFrame.from_dict(rows)
    col_index = pd.MultiIndex.from_tuples(col_tuples, names=["System", "Coords"])
    row_index = pd.MultiIndex.from_tuples(row_tuples, names=["Shape", "Verts"])
    df.index = row_index
    df.drop(columns="name", axis=1, inplace=True)
    df.columns = col_index
    return df

In [16]:
tet = tabulate(fx.Tetrahedron())
invtet = tabulate(fx.InvTetrahedron())
cu   = tabulate(fx.Cube())
octa = tabulate(fx.Octahedron())
rd   = tabulate(fx.RD())
co   = tabulate(fx.Cuboctahedron())

master_df = pd.concat((tet, invtet, cu, octa, rd, co), axis=0)
master_df

System      IVM                  XYZ                        
Coords        A  B  C  D           X           Y           Z
Shape Verts                                                 
Tet   a       1  0  0  0   sqrt(2)/4   sqrt(2)/4   sqrt(2)/4
      b       0  1  0  0  -sqrt(2)/4  -sqrt(2)/4   sqrt(2)/4
      c       0  0  1  0  -sqrt(2)/4   sqrt(2)/4  -sqrt(2)/4
      d       0  0  0  1   sqrt(2)/4  -sqrt(2)/4  -sqrt(2)/4
~Tet  e       0  1  1  1  -sqrt(2)/4  -sqrt(2)/4  -sqrt(2)/4
      f       1  0  1  1   sqrt(2)/4   sqrt(2)/4  -sqrt(2)/4
      g       1  1  0  1   sqrt(2)/4  -sqrt(2)/4   sqrt(2)/4
      h       1  1  1  0  -sqrt(2)/4   sqrt(2)/4   sqrt(2)/4
Cube  a       1  0  0  0   sqrt(2)/4   sqrt(2)/4   sqrt(2)/4
      b       0  1  0  0  -sqrt(2)/4  -sqrt(2)/4   sqrt(2)/4
      c       0  0  1  0  -sqrt(2)/4   sqrt(2)/4  -sqrt(2)/4
      d       0  0  0  1   sqrt(2)/4  -sqrt(2)/4  -sqrt(2)/4
      e       0  1  1  1  -sqrt(2)/4  -sqrt(2)/4  -sqrt(2)/4
      f       1  0  1  1   sqrt(2)/4   sqrt(2)/4  -sqrt(2)/4
      g       1  1  0  1   sqrt(2)/4  -sqrt(2)/4   sqrt(2)/4
      h       1  1  1  0  -sqrt(2)/4   sqrt(2)/4   sqrt(2)/4
Oct   i       1  1  0  0           0           0   sqrt(2)/2
      j       1  0  1  0           0   sqrt(2)/2           0
      k       1  0  0  1   sqrt(2)/2           0           0
      l       0  1  1  0  -sqrt(2)/2           0           0
      m       0  1  0  1           0  -sqrt(2)/2           0
      n       0  0  1  1           0           0  -sqrt(2)/2
RD    a       1  0  0  0   sqrt(2)/4   sqrt(2)/4   sqrt(2)/4
      b       0  1  0  0  -sqrt(2)/4  -sqrt(2)/4   sqrt(2)/4
      c       0  0  1  0  -sqrt(2)/4   sqrt(2)/4  -sqrt(2)/4
      d       0  0  0  1   sqrt(2)/4  -sqrt(2)/4  -sqrt(2)/4
      e       0  1  1  1  -sqrt(2)/4  -sqrt(2)/4  -sqrt(2)/4
      f       1  0  1  1   sqrt(2)/4   sqrt(2)/4  -sqrt(2)/4
      g       1  1  0  1   sqrt(2)/4  -sqrt(2)/4   sqrt(2)/4
      h       1  1  1  0  -sqrt(2)/4   sqrt(2)/4   sqrt(2)/4
      i       1  1  0  0           0           0   sqrt(2)/2
      j       1  0  1  0           0   sqrt(2)/2           0
      k       1  0  0  1   sqrt(2)/2           0           0
      l       0  1  1  0  -sqrt(2)/2           0           0
      m       0  1  0  1           0  -sqrt(2)/2           0
      n       0  0  1  1           0           0  -sqrt(2)/2
CO    o       2  1  1  0           0   sqrt(2)/2   sqrt(2)/2
      p       2  1  0  1   sqrt(2)/2           0   sqrt(2)/2
      q       1  2  1  0  -sqrt(2)/2           0   sqrt(2)/2
      r       1  2  0  1           0  -sqrt(2)/2   sqrt(2)/2
      s       1  0  2  1           0   sqrt(2)/2  -sqrt(2)/2
      t       1  0  1  2   sqrt(2)/2           0  -sqrt(2)/2
      u       0  1  2  1  -sqrt(2)/2           0  -sqrt(2)/2
      v       0  1  1  2           0  -sqrt(2)/2  -sqrt(2)/2
      w       1  1  2  0  -sqrt(2)/2   sqrt(2)/2           0
      x       0  2  1  1  -sqrt(2)/2  -sqrt(2)/2           0
      y       1  1  0  2   sqrt(2)/2  -sqrt(2)/2           0
      z       2  0  1  1   sqrt(2)/2   sqrt(2)/2           0

In [17]:
df = tabulate(fx.Icosahedron(),dec=True,n=5)
df

System           IVM                                  XYZ                    
Coords             A        B        C        D         X         Y         Z
Shape Verts                                                                  
Ico   o       1.8512  0.70711   1.1441        0       0.0   0.80902   0.50000
      p       1.8512   1.1441        0  0.70711   0.50000       0.0   0.80902
      q       1.1441   1.8512  0.70711        0  -0.50000       0.0   0.80902
      r      0.70711   1.8512        0   1.1441       0.0  -0.80902   0.50000
      s       1.1441        0   1.8512  0.70711       0.0   0.80902  -0.50000
      t      0.70711        0   1.1441   1.8512   0.50000       0.0  -0.80902
      u            0  0.70711   1.8512   1.1441  -0.50000       0.0  -0.80902
      v            0   1.1441  0.70711   1.8512       0.0  -0.80902  -0.50000
      w      0.70711   1.1441   1.8512        0  -0.80902   0.50000       0.0
      x            0   1.8512   1.1441  0.70711  -0.80902  -0.50000       0.0
      y       1.1441  0.70711        0   1.8512   0.80902  -0.50000       0.0
      z       1.8512        0  0.70711   1.1441   0.80902   0.50000       0.0

In [18]:
fx.Cuboctahedron().edges

[Edge from ivm_vector(a=1, b=2, c=1, d=0) to ivm_vector(a=0, b=2, c=1, d=1),
 Edge from ivm_vector(a=1, b=0, c=1, d=2) to ivm_vector(a=0, b=1, c=1, d=2),
 Edge from ivm_vector(a=0, b=1, c=2, d=1) to ivm_vector(a=1, b=1, c=2, d=0),
 Edge from ivm_vector(a=0, b=1, c=1, d=2) to ivm_vector(a=1, b=1, c=0, d=2),
 Edge from ivm_vector(a=2, b=1, c=1, d=0) to ivm_vector(a=1, b=1, c=2, d=0),
 Edge from ivm_vector(a=0, b=1, c=2, d=1) to ivm_vector(a=0, b=1, c=1, d=2),
 Edge from ivm_vector(a=2, b=1, c=1, d=0) to ivm_vector(a=1, b=2, c=1, d=0),
 Edge from ivm_vector(a=2, b=1, c=1, d=0) to ivm_vector(a=2, b=1, c=0, d=1),
 Edge from ivm_vector(a=2, b=1, c=0, d=1) to ivm_vector(a=2, b=0, c=1, d=1),
 Edge from ivm_vector(a=1, b=0, c=1, d=2) to ivm_vector(a=2, b=0, c=1, d=1),
 Edge from ivm_vector(a=1, b=2, c=1, d=0) to ivm_vector(a=1, b=2, c=0, d=1),
 Edge from ivm_vector(a=1, b=0, c=2, d=1) to ivm_vector(a=1, b=0, c=1, d=2),
 Edge from ivm_vector(a=1, b=0, c=2, d=1) to ivm_vector(a=1, b=1, c=2, d=0),

In [19]:
fx.Cuboctahedron().faces

(('o', 'w', 's', 'z'),
 ('z', 'p', 'y', 't'),
 ('t', 'v', 'u', 's'),
 ('w', 'q', 'x', 'u'),
 ('o', 'p', 'r', 'q'),
 ('r', 'y', 'v', 'x'),
 ('z', 's', 't'),
 ('t', 'y', 'v'),
 ('y', 'p', 'r'),
 ('r', 'q', 'x'),
 ('x', 'u', 'v'),
 ('u', 's', 'w'),
 ('w', 'q', 'o'),
 ('o', 'z', 'p'))

In [20]:
the_edges = fx.Cuboctahedron().unique
the_edges

{('o', 'p'),
 ('o', 'q'),
 ('o', 'w'),
 ('o', 'z'),
 ('p', 'r'),
 ('p', 'y'),
 ('p', 'z'),
 ('q', 'r'),
 ('q', 'w'),
 ('q', 'x'),
 ('r', 'x'),
 ('r', 'y'),
 ('s', 't'),
 ('s', 'u'),
 ('s', 'w'),
 ('s', 'z'),
 ('t', 'v'),
 ('t', 'y'),
 ('t', 'z'),
 ('u', 'v'),
 ('u', 'w'),
 ('u', 'x'),
 ('v', 'x'),
 ('v', 'y')}

In [21]:
labels = set()
for verts in the_edges:
    labels.add(verts[0])
    labels.add(verts[1])
labels = sorted(list(labels))

In [22]:
labels

['o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [23]:
face_matrix = pd.DataFrame(data=np.zeros((len(labels), len(labels)), dtype=int), 
                           index = labels,
                           columns = labels)
face_matrix

,o,p,q,r,s,t,u,v,w,x,y,z
o,0,0,0,0,0,0,0,0,0,0,0,0
p,0,0,0,0,0,0,0,0,0,0,0,0
q,0,0,0,0,0,0,0,0,0,0,0,0
r,0,0,0,0,0,0,0,0,0,0,0,0
s,0,0,0,0,0,0,0,0,0,0,0,0
t,0,0,0,0,0,0,0,0,0,0,0,0
u,0,0,0,0,0,0,0,0,0,0,0,0
v,0,0,0,0,0,0,0,0,0,0,0,0
w,0,0,0,0,0,0,0,0,0,0,0,0
x,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
for verts in the_edges:
    face_matrix.loc[verts[0], verts[1]] = 1
    face_matrix.loc[verts[1], verts[0]] = 1
face_matrix

,o,p,q,r,s,t,u,v,w,x,y,z
o,0,1,1,0,0,0,0,0,1,0,0,1
p,1,0,0,1,0,0,0,0,0,0,1,1
q,1,0,0,1,0,0,0,0,1,1,0,0
r,0,1,1,0,0,0,0,0,0,1,1,0
s,0,0,0,0,0,1,1,0,1,0,0,1
t,0,0,0,0,1,0,0,1,0,0,1,1
u,0,0,0,0,1,0,0,1,1,1,0,0
v,0,0,0,0,0,1,1,0,0,1,1,0
w,1,0,1,0,1,0,1,0,0,0,0,0
x,0,0,1,1,0,0,1,1,0,0,0,0


In [25]:
def make_matrix(shape):
    the_edges = shape.unique
    labels = set()
    for verts in the_edges:
        labels.add(verts[0])
        labels.add(verts[1])
    labels = sorted(list(labels))
    face_matrix = pd.DataFrame(data=np.zeros((len(labels), len(labels)), dtype=int), 
                           index = labels,
                           columns = labels)
    for verts in the_edges:
        face_matrix.loc[verts[0], verts[1]] = 1
        face_matrix.loc[verts[1], verts[0]] = 1
    return face_matrix

In [26]:
make_matrix(fx.Tetrahedron())

,a,b,c,d
a,0,1,1,1
b,1,0,1,1
c,1,1,0,1
d,1,1,1,0


### Testing Area 51

In [27]:
df.loc[('Ico', 'z')]['XYZ'].Y.evalf(5).round(8)

0.50000

In [28]:
control = (fx.Z - fx.T).length()
half = fx.half
midface = (fx.Z + fx.Y)
gold    = half * fx.PHI * midface/midface.length()
Zi = gold + fx.J/fx.J.length() * (control/2)
Yi = gold + fx.M/fx.M.length() * (control/2)

In [29]:
control/2

1/2

In [30]:
gold + fx.J/fx.J.length() * (control/2)

ivm_vector(a=sqrt(2)/2 + sqrt(2)*(1/2 + sqrt(5)/2)/2, b=0, c=sqrt(2)/2, d=sqrt(2)*(1/2 + sqrt(5)/2)/2)

In [31]:
half

1/2

In [32]:
type(midface.norm0()[0])

sympy.core.numbers.One

In [33]:
gold

ivm_vector(a=sqrt(2)*(1/2 + sqrt(5)/2)/2, b=0, c=0, d=sqrt(2)*(1/2 + sqrt(5)/2)/2)

In [34]:
Zi

ivm_vector(a=sqrt(2)/2 + sqrt(2)*(1/2 + sqrt(5)/2)/2, b=0, c=sqrt(2)/2, d=sqrt(2)*(1/2 + sqrt(5)/2)/2)

In [35]:
Yi

ivm_vector(a=sqrt(2)*(1/2 + sqrt(5)/2)/2, b=sqrt(2)/2, c=0, d=sqrt(2)/2 + sqrt(2)*(1/2 + sqrt(5)/2)/2)

In [36]:
fx.M.length()

sqrt(2)/2

In [37]:
fx.A

ivm_vector(a=1, b=0, c=0, d=0)

In [38]:
type(fx.A.a)

sympy.core.numbers.One

In [39]:
tv.Svol.simplify() # S module volume

16/(1 + sqrt(5))**5

In [40]:
tv.Evol.simplify()  # E module volume

sqrt(2)/(1 + sqrt(5))**3

In [41]:
Sfactor = tv.Svol/tv.Evol
Sfactor.simplify()

-sqrt(10) + 3*sqrt(2)

In [42]:
cuboctavol, icosavol = fx.Cuboctahedron().volume, fx.Icosahedron().volume 

In [43]:
cuboctavol, icosavol.evalf(50)

(20, 18.512295868219161196009899292654531923571426913640)

In [44]:
cuboctavol/icosavol == Sfactor

True

In [45]:
IW = (5*half * Sfactor ** 2).simplify() # IW = 2.5 CO * sfactor * sfactor
IW

320/(1 + sqrt(5))**4

In [46]:
IW.evalf(50)

2.9179606750063091077247899380617129367814492116542

In [47]:
(IW + 24 * tv.Svol).simplify()

4

## Scratch Pad for Facebook

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/54237268265/in/dateposted/" title="Crow Town"><img src="https://live.staticflickr.com/65535/54237268265_393966954d.jpg" width="495" height="500" alt="Crow Town"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/54237184569/in/dateposted/" title="XYZ IVM conversion"><img src="https://live.staticflickr.com/65535/54237184569_f7a02761c8.jpg" width="500" height="489" alt="XYZ IVM conversion"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/54236059212/in/dateposted/" title="Screen Shot 2024-12-30 at 6.04.21 PM"><img src="https://live.staticflickr.com/65535/54236059212_53a6aff839_o.png" width="210" height="236" alt="Screen Shot 2024-12-30 at 6.04.21 PM"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

In [48]:
r_edge_cube = {
'big_yellow' : fx.Vector((0,0,0)),
'small_yellow' : fx.Vector((1,1,1)),
'small_blue' : fx.Vector((1,0,0)),
'big_blue' : fx.Vector((0,1,1)),
'small_green' : fx.Vector((0,1,0)),
'big_green' : fx.Vector((1,0,1)),
'small_red' : fx.Vector((0,0,1)),
'big_red' : fx.Vector((1,1,0)),
}

In [49]:
by = r_edge_cube['big_yellow']

In [50]:
by.xyz

xyz_vector(x=0, y=0, z=0)

In [51]:
type(by)

qrays.Vector

In [52]:
sy = r_edge_cube['small_yellow']

In [53]:
sy.length()

sqrt(3)/2

In [54]:
bb = r_edge_cube['big_blue']

In [55]:
(bb-by).length()

sqrt(2)/2

In [56]:
sr = r_edge_cube['small_red']

In [57]:
(by - sr).length()

1/2

In [58]:
print(sympy.sqrt(sympy.Rational(9,8)).evalf(10))

1.060660172


In [59]:
X = fx.Vector((1,0,0))
X.length()

1/2

In [60]:
A = fx.A
A.length()

sqrt(6)/4

In [61]:
(fx.A - fx.B).length()

1

In [62]:
A.xyz.length()

sqrt(6)/8

In [63]:
A.length()

sqrt(6)/4

In [64]:
X.quadray().length()

1

In [65]:
X.length()

1/2

In [66]:
(fx.A.xyz - fx.B.xyz).length()

1/2

In [67]:
br = r_edge_cube['big_red']
bg = r_edge_cube['big_green']
sb = r_edge_cube['small_blue']

In [68]:
(br - bg).length()

sqrt(2)/2

In [69]:
(br - sb).length()

1/2